### **GTACG analysis**

In [ ]:
# to specific details about the input files see:
# https://github.com/caiorns/GTACG-backend

In [1]:
# 1. list all CDS
dic_file  = 'input_genomes.dic'
input_faa = 'input_genomes.faa'
print('java -jar GTACG.jar ExportAllSequences -dic '+dic_file+' -dicFormat faa -out '+input_faa)

java -jar GTACG.jar ExportAllSequences -dic input_genomes.dic -dicFormat faa -out input_genomes.faa


In [2]:
# 2. blast and zip results

input_db = 'blast/'+input_faa+'.db'

print('makeblastdb -dbtype prot -in '+input_faa+' -out '+input_db+'\n')
out_blast = input_faa+'.m8'

print('cat '+input_faa+' | parallel --block 50k -j 10 --recstart "\n>" --pipe '+
      'blastp -evalue 0.0000000001 -outfmt 6 -db '+input_db+' -num_alignments 1000 -query - > '+out_blast,'\n')

print('gzip '+out_blast)

makeblastdb -dbtype prot -in input_genomes.faa -out blast/input_genomes.faa.db

cat input_genomes.faa | parallel --block 50k -j 10 --recstart "
>" --pipe blastp -evalue 0.0000000001 -outfmt 6 -db blast/input_genomes.faa.db -num_alignments 1000 -query - > input_genomes.faa.m8 

gzip input_genomes.faa.m8


In [3]:
# 3. ExploratoryClustering
blast_gz = out_blast+'.gz'
print('java -jar GTACG.jar ExploratoryClustering -dic '+dic_file+' -dicFormat faa -m8 '+blast_gz+
      ' -start 45 -end 95 -threads 10 -interval 10')

java -jar GTACG.jar ExploratoryClustering -dic input_genomes.dic -dicFormat faa -m8 input_genomes.faa.m8.gz -start 45 -end 95 -threads 10 -interval 10


In [5]:
# 4. ExportGraph
rests_file = '45.rests'
graph      = 'Xylella_45rests.graph'
heads_file = 'Xylella_45rests.heads'
print('java -jar GTACG.jar ExportGraph -dic '+dic_file+' -dicFormat faa -m8 '+blast_gz+
      ' -rests '+rests_file+' -out '+graph+' -heads '+heads_file)

java -jar GTACG.jar ExportGraph -dic input_genomes.dic -dicFormat faa -m8 input_genomes.faa.m8.gz -rests 45.rests -out Xylella_45rests.graph -heads Xylella_45rests.heads


In [6]:
# 5. Compressing graph file
graph_file = graph+'.gz'
print('gzip '+graph)

gzip Xylella_45rests.graph


In [7]:
# 6. ExportTreeFiles
trees_folder = 'Xylella_45rests_trees'
print('java -jar GTACG.jar ExportTreeFiles -dic '+dic_file+' -dicFormat faa -graph '+graph_file+
      ' -heads '+heads_file+' -align clustalO -tree fastTree -threads 10 -out '+trees_folder)

java -jar GTACG.jar ExportTreeFiles -dic input_genomes.dic -dicFormat faa -graph Xylella_45rests.graph.gz -heads Xylella_45rests.heads -align clustalO -tree fastTree -threads 10 -out Xylella_45rests_trees


In [8]:
# 7. ExportReport (core alignment)
groups     = 'country.group host.group st.group'
data_folder_CoreAln = 'Xylella_45rests_CoreAln'
print('java -jar GTACG.jar ExportReport -dic '+dic_file+' -dicFormat faa -graph '+graph_file+
      ' -heads '+heads_file+' -trees '+trees_folder+' -out '+data_folder_CoreAln+' -threads 10'+
      ' -printCoreAlignment ')

java -jar GTACG.jar ExportReport -dic input_genomes.dic -dicFormat faa -graph Xylella_45rests.graph.gz -heads Xylella_45rests.heads -trees Xylella_45rests_trees -out Xylella_45rests_CoreAln -threads 10 -printCoreAlignment 


In [11]:
# 8. ExportReport (data)
data_folder = 'Xylella_45rests_data'
print('java -jar GTACG.jar ExportReport -dic '+dic_file+' -dicFormat faa -graph '+graph_file+
      ' -heads '+heads_file+' -trees '+trees_folder+' -groups '+groups+' -out '+data_folder+' -threads 10')

java -jar GTACG.jar ExportReport -dic input_genomes.dic -dicFormat faa -graph Xylella_45rests.graph.gz -heads Xylella_45rests.heads -trees Xylella_45rests_trees -groups country.group host.group st.group -out Xylella_45rests_data -threads 10


### **Iq-tree analysis** 

In [12]:
iqtree_command = 'iqtree-omp -s core_amino.align -bb 1000 -nt AUTO -m TESTMERGE'
print(iqtree_command)

iqtree-omp -s core_amino.align -bb 1000 -nt AUTO -m TESTMERGE


### **COG analysis** 

In [13]:
query  = 'genome_Xf1.faa'
output = 'genome_Xf1.rpsblast'
rpsblast_line = 'rpsblast+ -db cog_database -query '+query+' -out '+output+' -num_threads 10 -evalue 0.000001 -max_target_seqs 5 -max_hsps 1 -outfmt "6 qseqid sseqid pident qcovs bitscore score evalue"'
print(rpsblast_line)

rpsblast+ -db cog_database -query genome_Xf1.faa -out genome_Xf1.rpsblast -num_threads 10 -evalue 0.000001 -max_target_seqs 5 -max_hsps 1 -outfmt "6 qseqid sseqid pident qcovs bitscore score evalue"
